# Building a Spam Filter with Naive Bayes

n this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The UCI Machine Learning Repository. The data collection process is described in more details on this page (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Exploring the Dataset

In [2]:
# Read the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label','SMS'])
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Number of rows and columns
sms_spam.shape

(5572, 2)

In [4]:
# Message distribution
sms_spam['Label'].value_counts(normalize=True) # percentage format (spam vs ham - non-spam)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

About 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

## Training and Test Set

In [5]:
# Split data in training and test sets as 80% and 20%
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1,          # randomize the entire dataset
                                  random_state=1)  # ensure results are reproducible

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print('Number of rows and columns of train set: ' + str(training_set.shape))
print('Number of rows and columns of train set: ' + str(test_set.shape))

Number of rows and columns of train set: (4458, 2)
Number of rows and columns of train set: (1114, 2)


We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [6]:
print('Train Set: \n' + str(training_set['Label'].value_counts(normalize=True)))
print('Test Set: \n' + str(test_set['Label'].value_counts(normalize=True)))

Train Set: 
ham     0.86541
spam    0.13459
Name: Label, dtype: float64
Test Set: 
ham     0.868043
spam    0.131957
Name: Label, dtype: float64


The results look good! We'll now move on to cleaning the dataset.

## Data Cleaning

Our Naive Bayes algorithm will make the classification based on the results it gets to these two equations (note that we replaced "SpamC" with "Ham" inside the second equation below):

$$P(Spam|w1,w2,...,wn) ∝ P(Spam).∏_{i=1}^nP(w_{i}|Spam)$$
$$P(Ham|w1,w2,...,wn) ∝ P(Ham).∏_{i=1}^nP(w_{i}|Ham) $$

To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

$$P(w_{i}|Spam) = (N_{w_{i}|Spam} + α) / (N_{Spam} + α.N_{Vocabulary})$$
$$P(w_{i}|Ham) = (N_{w_{i}|Ham} + α) / (N_{Ham} + α.N_{Vocabulary})$$

whereas:

$N_{w_{i}|Spam}$ = number of times the word $w_{i}$ occurs in spam messages

$N_{w_{i}|Spam^C}$ = number of times the word $w_{i}$ occurs in non-spam messages

$N_{Spam}$ = total number of words in spam messages

$N_{Spam^C}$ = total number of words in non-spam messages

$N_{Vocabulary}$ = total number of words in the vocabulary

α = 1 (α is a smoothing parameter)

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need:

- The SMS column doesn't exist anymore.
- Instead, the SMS column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row describes a single message. For instance, the first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values spam, 2, 2, 1, 1, 0, 0, 0, 0, 0. These values tell us that:
    - The message is spam.
    - The word "secret" occurs two times inside the message.
    - The word "prize" occurs two times inside the message.
    - The word "claim" occurs one time inside the message.
    - The word "now" occurs one time inside the message.
    - The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

## Letter Case and Punctuation

In [7]:
# Before cleaning
training_set['SMS'].head()

0                         Yep, by the pretty sculpture
1        Yes, princess. Are you going to make me moan?
2                           Welp apparently he retired
3                                              Havent.
4    I forgot 2 ask ü all smth.. There's a card on ...
Name: SMS, dtype: object

In [8]:
# Strip punctuation from any character not from a-z, A-Z or 0-9
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ') 

# Bring all words to lower case
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating the Vocabulary

In [9]:
# Create the vocabulary (a list with all the unique words in our training set)
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = [word for sms in training_set['SMS'] for word in sms]
# vocabulary = []
# for sms in training_set['SMS']:
#     for word in sms:
#         vocabulary.append(word)

# Turn vocabulary into a set to remove the duplicates from the list then set it back into a list
vocabulary = list(set(vocabulary))
print('Number of unique words in all the messages of training set: '+str(len(vocabulary)))

Number of unique words in all the messages of training set: 7783


In [10]:
vocabulary.sort()

## The Final Training Set

In [11]:
# Create a dictionary for training set (, )
word_counts_per_sms = {unique_word: [0]           # key: a unique word from the vocabulary
                       * len(training_set['SMS']) # value: a list of the length of training set
                       for unique_word in vocabulary}

# Loop through SMS to get both the index and message
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [12]:
# Create a dataframe for the dictionary
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [13]:
# Final training data
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

- P(Spam) and P(Ham)
- $N_{Spam}, N_{Ham}, N_{Vocabulary}$

We'll also use Laplace smoothing and set α=1.

In [14]:
# Create spam and ham sets
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']
spam_messages.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
16,spam,"[freemsg, why, haven, t, you, replied, to, my,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,spam,"[congrats, 2, mobile, 3g, videophones, r, your...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,spam,"[free, message, activate, your, 500, free, tex...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,spam,"[call, from, 08702490080, tells, u, 2, call, 0...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,spam,"[someone, has, conacted, our, dating, service,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)
print('P(Spam) = '+str(p_spam))
print('P(Ham) = '+str(p_ham))

P(Spam) = 0.13458950201884254
P(Ham) = 0.8654104979811574


In [16]:
# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()
print('N_Spam = '+str(n_spam))

N_Spam = 15190


In [17]:
# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()
print('N_Ham = '+str(n_ham))

N_Ham = 57237


In [18]:
# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

print('N_Vocabulary = '+str(n_vocabulary))

N_Vocabulary = 7783


## Calculating Parameters

In [19]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages 
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam       # P(wi|Spam)
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages 
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham       # P(wi|Ham)

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message $(w_{1}, w_{2}, ..., w_{n})$.
- Calculates P(Spam|$w_{1}, w_{2}, ..., w_{n}$) and P(Ham|$w_{1}, w_{2}, ..., w_{n}$).
- Compares the values of P(Spam|$w_{1}, w_{2}, ..., w_{n}$) and P(Ham|$w_{1}, w_{2}, ..., w_{n}$), and:
   - If P(Ham|$w_{1}, w_{2}, ..., w_{n}$) > P(Spam|$w_{1}, w_{2}, ..., w_{n}$), then the message is classified as ham.
   - If P(Ham|$w_{1}, w_{2}, ..., w_{n}$) < P(Spam|$w_{1}, w_{2}, ..., w_{n}$), then the message is classified as spam.
   - If P(Ham|$w_{1}, w_{2}, ..., w_{n}$) = P(Spam|$w_{1}, w_{2}, ..., w_{n}$), then the algorithm may request human help.

In [20]:
import re

# Create a function for two new messages
def classify(message):
    '''
    message: a string
    '''
    # Replace punctuation, make all letters lowercase then split each word
    message = re.sub('\W', ' ', message)
    message = message.lower().split() # turn into a list
    
    # Initiate values
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Iterate over each word in message
    for word in message:
        
        # If word is present
        # Update the p-value by multiplying with the parameter value of that word
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [21]:
# Test some messages
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [22]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set of 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [26]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [27]:
# Create a new column for the label
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [28]:
# Measure the accuracy of our spam filter to see how well our spam filter does

correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

## Next Steps

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

## Next steps include:

- Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
- Make the filtering process more complex by making the algorithm sensitive to letter case